In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "https://news.ycombinator.com/"
headers = {'User-Agent': 'Mozilla/5.0'}
res = requests.get(url, headers=headers)
soup = BeautifulSoup(res.text, "html.parser")

results = []

articles = soup.find_all("tr", class_="athing")

for article in articles:
    rank_tag = article.find("span", class_="rank")
    title_tag = article.find("a", class_="titlelink")

    if not rank_tag or not title_tag:
        continue  # skip broken items

    rank = rank_tag.get_text(strip=True).replace('.', '')
    title = title_tag.get_text(strip=True)
    link = title_tag["href"]

    subtext_row = article.find_next_sibling("tr")
    subtext = subtext_row.find("td", class_="subtext") if subtext_row else None

    if not subtext:
        continue

    score_tag = subtext.find("span", class_="score")
    score = score_tag.get_text(strip=True).replace(" points", "") if score_tag else "0"

    author_tag = subtext.find("a", class_="hnuser")
    author = author_tag.get_text(strip=True) if author_tag else "Unknown"

    comment_links = subtext.find_all("a")
    comments_text = comment_links[-1].get_text(strip=True) if comment_links else "0"
    comments = "0"
    if "comment" in comments_text:
        comments = ''.join(filter(str.isdigit, comments_text))

    results.append({
        "rank": int(rank),
        "title": title,
        "link": link,
        "score": int(score),
        "author": author,
        "comments": int(comments)
    })

df = pd.DataFrame(results)
df.to_csv("hn_top30.csv", index=False)
df.head()


""
